# Importing Required Libraries

In [50]:

import numpy as np
import pandas as pd
import difflib

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity



# Data Collection and Processing

Loading the movies dataset to pandas dataframe

In [37]:

movies_data = pd.read_csv('/content/movies.csv', engine='python', on_bad_lines='skip')



Printing the first 5 rows

In [38]:

movies_data.head()



,index,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,...,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,cast,crew,director
0,0,237000000,Action Adventure Fantasy Science Fiction,http://www.avatarmovie.com/,19995,culture clash future space war space colony so...,en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,...,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,Sam Worthington Zoe Saldana Sigourney Weaver S...,"[{'name': 'Stephen E. Rivkin', 'gender': 0, 'd...",James Cameron
1,1,300000000,Adventure Fantasy Action,http://disney.go.com/disneypictures/pirates/,285,ocean drug abuse exotic island east india trad...,en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,...,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500,Johnny Depp Orlando Bloom Keira Knightley Stel...,"[{'name': 'Dariusz Wolski', 'gender': 2, 'depa...",Gore Verbinski
2,2,245000000,Action Adventure Crime,http://www.sonypictures.com/movies/spectre/,206647,spy based on novel secret agent sequel mi6,en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,...,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466,Daniel Craig Christoph Waltz L\u00e9a Seydoux ...,"[{'name': 'Thomas Newman', 'gender': 2, 'depar...",Sam Mendes
3,3,250000000,Action Crime Drama Thriller,http://www.thedarkknightrises.com/,49026,dc comics crime fighter terrorist secret ident...,en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,...,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106,Christian Bale Michael Caine Gary Oldman Anne ...,"[{'name': 'Hans Zimmer', 'gender': 2, 'departm...",Christopher Nolan
4,4,260000000,Action Adventure Science Fiction,http://movies.disney.com/john-carter,49529,based on novel mars medallion space travel pri...,en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,...,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124,Taylor Kitsch Lynn Collins Samantha Morton Wil...,"[{'name': 'Andrew Stanton', 'gender': 2, 'depa...",Andrew Stanton


Printing the number of rows and columns

In [39]:

movies_data.shape



(4803, 24)

# Selecting Important Features

In [40]:

selected_features = ['genres','keywords','tagline','cast','director']
print(selected_features)



['genres', 'keywords', 'tagline', 'cast', 'director']


# Handling missing values

In [41]:

for feature in selected_features:
  movies_data[feature] = movies_data[feature].fillna('')



# Combining All Selected Features

In [42]:

combined_features = movies_data['genres']+' '+movies_data['keywords']+' '+movies_data['tagline']+' '+movies_data['cast']+' '+movies_data['director']

combined_features.head()



,0
0,Action Adventure Fantasy Science Fiction cultu...
1,Adventure Fantasy Action ocean drug abuse exot...
2,Action Adventure Crime spy based on novel secr...
3,Action Crime Drama Thriller dc comics crime fi...
4,Action Adventure Science Fiction based on nove...


# Converting Text Data into Feature Vectors (TF-IDF)

In [51]:

vectorizer = TfidfVectorizer(
    stop_words='english',
    max_features=5000
)

feature_vectors = vectorizer.fit_transform(combined_features)



# Calculating Cosine Similarity

In [47]:

similarity = cosine_similarity(feature_vectors)

similarity.shape


(4803, 4803)

# Creating Movie Recommendation Function

In [48]:

def recommend_movies(movie_name, num_recommendations=30):

    list_of_all_titles = movies_data['title'].tolist()

    find_close_match = difflib.get_close_matches(
        movie_name,
        list_of_all_titles
    )

    if not find_close_match:
        print("No movie found. Please check spelling.")
        return

    close_match = find_close_match[0]

    index_of_the_movie = movies_data[
        movies_data['title'] == close_match
    ].index[0]

    similarity_score = list(
        enumerate(similarity[index_of_the_movie])
    )

    sorted_similar_movies = sorted(
        similarity_score,
        key=lambda x: x[1],
        reverse=True
    )

    print(f"\nMovies suggested for you based on '{close_match}':\n")

    i = 1
    for movie in sorted_similar_movies[1:]:
        index = movie[0]
        title_from_index = movies_data.iloc[index]['title']

        if i <= num_recommendations:
            print(i, '.', title_from_index)
            i += 1



---
---

# **Movie Recommendation System**:

In [49]:

movie_name = input("Enter your favourite movie name: ")
recommend_movies(movie_name)



Enter your favourite movie name: Endgame

Movies suggested for you based on 'Ender's Game':

1 . Hugo
2 . Begin Again
3 . 3 Days to Kill
4 . The Helix... Loaded
5 . X-Men Origins: Wolverine
6 . Intolerance
7 . Pitch Perfect 2
8 . Oliver Twist
9 . BloodRayne
10 . Nanny McPhee and the Big Bang
11 . True Grit
12 . The Boy in the Striped Pyjamas
13 . Zambezia
14 . The Jungle Book
15 . Nim's Island
16 . The Empire Strikes Back
17 . Wicked Blood
18 . Tsotsi
19 . Shutter Island
20 . The Ultimate Gift
21 . Signs
22 . Rendition
23 . Maggie
24 . The Walk
25 . Return of the Jedi
26 . Split Second
27 . Ramanujan
28 . The Royal Tenenbaums
29 . Patriot Games
30 . The Conversation
